# Working With TiledRasterLayers

TiledRasterRDD will be the class that will see the most use. It provides all the methods needed to perform a computations and analysis on the data. When reading and saving layers, this class will be used.

This following guide will go over the creation methods and abilities unique to TiledRasterLayer. For RasterLayer, please see this [guide] and for a more general overview of layers, look [here].

## Setting up the Environment for This Guide

In [1]:
import numpy as np
import pyproj
from shapely.geometry import Point, Polygon, MultiPoint

import geopyspark as gps
from pyspark import SparkContext

In [2]:
conf = gps.geopyspark_conf(master="local[*]", appName="tiled-raster-layer-examples")
pysc = SparkContext(conf=conf)

## Creating TiledRasterLayers

### From PySpark RDD

Like `RasterLayer`s, `TiledRasterLayer`s can be created from RDDS using `from_numpy_rdd` as well. What is different, however, is `Metadata` which must also be passed in during initialization. This makes creating `TiledRasterLayer`s this way a little bit more arduous.

The following example constructs a RDD with a tuple. The first element is a `SpatialKey` because we have decidec to make the data spatial. If we were dealing with spatial-temproal data, then `SpaceTimeKey` would be the first element. `Tile` will always be the second element of the tuple.

In [4]:
spatial_key = gps.SpatialKey

data = np.array([[
    [1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 0.0]]], dtype=float)

tile = gps.Tile.from_numpy_array(numpy_array=data, no_data_value=-1.0)

layer = [(spatial_key(row=0, col=0), tile),
         (spatial_key(row=1, col=0), tile),
         (spatial_key(row=0, col=1), tile),
         (spatial_key(row=1, col=1), tile)]

rdd = pysc.parallelize(layer)

extent = gps.Extent(0.0, 0.0, 33.0, 33.0)
layout = gps.TileLayout(2, 2, 5, 5)
bounds = gps.Bounds(spatial_key(col=0, row=0), spatial_key(col=1, row=1))
layout_definition = gps.LayoutDefinition(extent, layout)

metadata = gps.Metadata(
    bounds=bounds,
    crs='+proj=longlat +datum=WGS84 +no_defs ',
    cell_type='float32ud-1.0',
    extent=extent,
    layout_definition=layout_definition)

tiled_layer = gps.TiledRasterLayer.from_numpy_rdd(pysc=pysc, layer_type=gps.LayerType.SPATIAL, numpy_rdd=rdd,
                                                  metadata=metadata)
tiled_layer

TiledRasterLayer(layer_type=LayerType.SPATIAL, zoom_level=None, is_floating_point_layer=True)

### Through Rasterization

Another means of producing `TiledRasterLayer` is through rasterizing a Shapely geometry via the `rasterize` method.

In [11]:
extent = gps.Extent(0.0, 0.0, 11.0, 11.0)
polygon = Polygon([(0, 11), (11, 11), (11, 0), (0, 0)])

gps.rasterize(pysc=pysc, geoms=[polygon], crs="EPSG:3857", zoom=10, fill_value=1.0)

TiledRasterLayer(layer_type=LayerType.SPATIAL, zoom_level=10, is_floating_point_layer=True)

### Through Euclidean Distance

The final way to create `TiledRasterLayer` is by calculating the Euclidean distance of a Shapely geometry. `euclidean_distance` is the class method which does this. While you can use any geometry to perform Euclidean distance, it is recommended not to use Polygons if they cover many cells of the resulting raster. As this can impact performance in a negative way.

In [12]:
latlong = pyproj.Proj(init='epsg:4326')
webmerc = pyproj.Proj(init='epsg:3857')
points = MultiPoint([pyproj.transform(latlong, webmerc, 1, 1),
                     pyproj.transform(latlong, webmerc, 2, 2)])

points = MultiPoint([Point(0, 0), Point(0, 1), Point(1, 0), Point(1, 1)])

# Makes a TiledRasterRDD from the Euclidean distance calculation.
gps.euclidean_distance(pysc=pysc, geometry=points, source_crs=3857, zoom=7)

TiledRasterLayer(layer_type=LayerType.SPATIAL, zoom_level=7, is_floating_point_layer=True)

## Using TiledRasterLayers

`TiledRasterLayer`s will be the class that will see the most use. It provides all the methods needed to perform a computations and analysis on the data. When reading and saving layers, this class will be used.

### Reprojecting

Often the tiles within a `TiledRasterLayer` will have to be reprojected. There is a method to do this aptly named, `reproject`. There are a few different ways to use this method. Each of which provides different results.

The first option is to specify an `Extent` and a `TileLayout`. Where the `Extent` is the area that will be covered by the tiles and the `TileLayout` describes the tiles and the grid they’re arranged on.

In [6]:
extent = gps.Extent(0.0, 0.0, 33.0, 33.0)
tile_layout = gps.TileLayout(2, 2, 5, 5)
layout_definition = gps.LayoutDefinition(extent, tile_layout)

tiled_layer.reproject(target_crs=3857, layout=layout_definition)

Py4JJavaError: An error occurred while calling o40.reproject.
: java.lang.RuntimeException: Cannot construct grid bounds of this size: 228652 x 234718
	at scala.sys.package$.error(package.scala:27)
	at geotrellis.raster.GridBounds.<init>(GridBounds.scala:78)
	at geotrellis.spark.tiling.MapKeyTransform.apply(MapKeyTransform.scala:76)
	at geotrellis.spark.TileLayerMetadata$.fromRdd(TileLayerMetadata.scala:172)
	at geotrellis.spark.package$withCollectMetadataMethods.collectMetadata(package.scala:190)
	at geotrellis.spark.reproject.TileRDDReproject$.apply(TileRDDReproject.scala:136)
	at geotrellis.spark.reproject.TileRDDReproject$.apply(TileRDDReproject.scala:238)
	at geopyspark.geotrellis.SpatialTiledRasterLayer.reproject(SpatialTiledRasterLayer.scala:89)
	at geopyspark.geotrellis.SpatialTiledRasterLayer.reproject(SpatialTiledRasterLayer.scala:49)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:6

The other option is to simply give `reproject` a `tile_size` that each tile should be in the resulting grid. `Extent` and `TileLayout` will be calculated from this size. Using this method will ensure that the native resolutions of the rasters are kept.

Below, we will be using the default `tile_size`, 256.

In [7]:
tiled_layer.reproject(target_crs=3857)

Py4JJavaError: An error occurred while calling o40.reproject.
: java.lang.RuntimeException: Cannot construct grid bounds of this size: 228652 x 234718
	at scala.sys.package$.error(package.scala:27)
	at geotrellis.raster.GridBounds.<init>(GridBounds.scala:78)
	at geotrellis.spark.tiling.MapKeyTransform.apply(MapKeyTransform.scala:76)
	at geotrellis.spark.TileLayerMetadata$.fromRdd(TileLayerMetadata.scala:172)
	at geotrellis.spark.package$withCollectMetadataMethods.collectMetadata(package.scala:190)
	at geotrellis.spark.reproject.TileRDDReproject$.apply(TileRDDReproject.scala:136)
	at geotrellis.spark.reproject.TileRDDReproject$.apply(TileRDDReproject.scala:238)
	at geotrellis.spark.reproject.TileRDDReprojectMethods.reproject(TileRDDReprojectMethods.scala:73)
	at geopyspark.geotrellis.SpatialTiledRasterLayer.reproject(SpatialTiledRasterLayer.scala:64)
	at geopyspark.geotrellis.SpatialTiledRasterLayer.reproject(SpatialTiledRasterLayer.scala:49)
	at sun.reflect.NativeMethodAccessorImpl.inv

If you're planning on making a TMS server, then the following is the only way to reproject the layer.

In [ ]:
tiled_layer.reproject(target_crs=3857, scheme=LayoutScheme.ZOOM)

### Retiling

It is possible to change the layout of the tiles within `TiledRasterLayer` via `tile_to_layout`.

In [ ]:
extent = Extent(100.0, 100.0, 250.0, 250.0)
tile_layout = TileLayout(5, 5, 256, 256)
layout_definition = LayoutDefinition(extent, tile_layout)

tiled_layer.tile_to_layout(layout=layout_definition)

### Masking

By using `mask`, the `TiledRasterRDD` can be masekd using one or more Shapely geometries.

In [ ]:
polygon = Polygon([(0, 0), (10, 0), (10, 10), (0, 10), (0, 0)])

tiled_layer.mask(geometries=polygon)

### Stitching

Using `stitch` will produce a single `Tile` by stitching together all of the tiles within the `TiledRasterLayer`. This can only be done with spatial layers, and is not recommended if the data contained within is large. As it can cause crashes due to its size.

In [ ]:
tiled_layer.stitch()

### Pyramiding

`pyramid` will create a new `TiledRasterLayer` for each zoom level, and the resulting list will then be used to create a new instance of the `Pyramid` class.

In [ ]:
tiled_layer.pyramid(start_zoom=12, end_zoom=1)

### Operations

TiledRasterRDDs can perform both local and focal operations.

#### Local

Performing local operations with `TiledRasterLayer`s can be performed with ints, floats, or other `TiledRasterLayer`s.

In [ ]:
tiled_layer + 1

(tiled_layer + tiled_layer) / 2

1 / (5 - tiled_layer)

#### Focal

Focal operations are done by selecting both a `neighborhood` and an `operation`. Because the inputs must be sent over to Scala, the operation must be entered in the form of a constant.

`neighborhood` can be specified with either a `Neighboorhod` sub-class, or a constant. The first example will be using one of the `Neighborhood` classes.

In [ ]:
square_neighborhood = Square(extent=1)
tiled_layer.focal(operation=Operation.SLOPE, neighborhood=square_neighborhood)

Performing a focal operation with a constant for `neighborhood`.

In [ ]:
tiled_layer.focal(operation=Operation.SLOPE, neighborhood=Neighborhood.SQUARE, param_1=1)

#### Polygonal Summary

In addition to local and focal methods, `TiledRasterLayer` can also perform polygonal summary methods. Using Shapely geometries, one can find the `min`, `max`, `sum`, and `mean` of all of the values intersected by the geometry.

In [8]:
polygon = Polygon([(0, 0), (10, 0), (10, 10), (0, 10), (0, 0)])

tiled_layer.polygonal_min(geometry=polygon, data_type=int)

tiled_layer.polygonal_max(geometry=polygon, data_type=float)

tiled_layer.polygonal_sum(geometry=polygon, data_type=int)

tiled_layer.polygonal_mean(geometry=polygon)

1.0